### 1.1 LDA Gibbs采样推导
#### 一. LDA中要用到的公式  
1. 完全条件公式 :  
 $$p\left( { x }_{ j }|{ x }_{ \neg j } \right) =\frac { p\left( { x }_{ 1 },{ x }_{ 2 },...,{ x }_{ n } \right)  }{ p\left( { x }_{ j }|{ x }_{ 1 },{ x }_{ 2 },...{ x }_{ j-1 },{ x }_{ j+1 },...,{ x }_{ n } \right)  } $$
2. $\varphi$ : $k*v$阶矩阵, ${ \overrightarrow { { \varphi  }_{ k } }  }$是主题$k$上的词分布  
$\theta$ : $k*m$阶矩阵, $\overrightarrow { { \theta  }_{ m } } $: 第$m$篇文章的主体分布
3. $\int { \prod _{ i=1 }^{ K }{ { \left( { p }_{ i } \right)  }^{ { \alpha  }_{ i }-1 } }  } d\overrightarrow { p } =\triangle \left( \overrightarrow { \alpha  }  \right) \\ \quad \quad \quad \quad \quad \quad =\frac { \prod _{ i=1 }^{ k }{ \Gamma \left( { \alpha  }_{ i } \right)  }  }{ \Gamma \left( \sum _{ i=1 }^{ k }{ { \alpha  }_{ i } }  \right)  } $

#### 二. LDA整个文本集生成的联合分布概率 - (词与每个次对应的主题)
由于$P\left( \overrightarrow { z } |\overrightarrow { w }  \right)$不好求, 因此使用`collapse gibbs sampling`抽样, 利用完全条件公式(1),写出$p\left( { z }_{ i }|\overrightarrow { { z }_{ \neg i } } ,\overrightarrow { w }  \right) $. 用此公式模拟$P\left( \overrightarrow { z } |\overrightarrow { w }  \right)$
1. 写出LDA下整个文本集生成的联合概率   
$$p\left( \overrightarrow { w } ,\overrightarrow { z } |\overrightarrow { \alpha  } ,\overrightarrow { \beta  }  \right) =p\left( \overrightarrow { w } |\overrightarrow { z } ,\overrightarrow { \beta  }  \right) *p\left( \overrightarrow { z } |\overrightarrow { \alpha  }  \right) $$ 
$\overrightarrow { w }$是整个语料库中出现的词向量, $\overrightarrow { z }$是这个语料库中每个单词属于的某个主题  
由于第一项不包含$\overrightarrow { \alpha  } $, 第二项不包括$\overrightarrow { \beta  } $, 所以可以分开处理  
  1. 第一项 : $$p\left( \overrightarrow { w } |\overrightarrow { z } ,\varphi  \right) =\prod _{ i=1 }^{ W }{ p\left( { w }^{ i }|{ z }^{ i } \right)  } =\prod _{ i=1 }^{ W }{ { \varphi  }_{ { z }_{ i },{ w }_{ i } } } $$其中,$W$是语料库中所有单词的数量, 这些词单词, 在$z_i=k$的条件下, 在多次独立的'多项分布'试验中被抽取出来.   
  ${ \varphi  }_{ { z }_{ i },{ w }_{ i } }$表示在主题${ z }_{ i }$下, 得到单词${ w }_{ i }$的概率  
  而当我们${ \varphi  }_{ { z }_{ i },{ w }_{ i } }$拆成两项, 一个`over topic`, 另一个`over vocabulary` , 就有$$p\left( \overrightarrow { w } |\overrightarrow { z } ,\varphi  \right) =\prod _{ i=1 }^{ W }{ p\left( { w }^{ i }|{ z }^{ i } \right)  } =\prod _{ k=1 }^{ K }{ \prod _{ \{ i,{ z }_{ i }=k\}  }^{ W }{ p\left( { w }_{ i }=t|{ z }_{ i }=k \right)  }  } =\prod _{ k=1 }^{ K }{ \prod _{ t=1 }^{ V }{ { { \varphi  }_{ k,t } }^{ { n }_{ k }^{ (t) } } }  } $$得到上式的一些解释 :   
    1.  ${ n }_{ k }^{ (t) }$为主题k下单词t被观测到的次数
    1. 为什么是2个连乘 : 因为该条件概率中, $\overrightarrow { z } $代表每个单词的主题概率分布, 即每个单词在所有K个主题下都会产生概率
    1. 我们没有考虑超参数$\overrightarrow { \beta  } $, 而是先假设已知所有主题的词分布$\varphi_i $, 然后利用超参数$\overrightarrow { \beta  } $与$\varphi_i $的关系, 把$\varphi$积分掉. 推到如下 
    2. $p\left( \overrightarrow { w } |\overrightarrow { z } ,\overrightarrow { \beta  }  \right) =\int { p\left( \overrightarrow { w } |\overrightarrow { z } ,\varphi  \right) *p\left( \varphi |\overrightarrow { \beta  }  \right)  } d\varphi \\ \quad \quad \quad \quad \quad =\int { p\left( \overrightarrow { w } |\overrightarrow { z } ,\varphi  \right) *Dirichlet\left( \varphi |\overrightarrow { \beta  }  \right)  } d\varphi \\ \quad \quad \quad \quad \quad =\int { \prod _{ k=1 }^{ K }{ \prod _{ t=1 }^{ V }{ { { \varphi  }_{ k,t } }^{ { n }_{ k }^{ (t) } } }  } *\prod _{ k=1 }^{ K }{ \left( \frac { 1 }{ \triangle ({ \overrightarrow { \beta  }  }) } \prod _{ t=1 }^{ V }{ { { \varphi  }_{ k,t } }^{ { \beta  }_{ t }-1 } }  \right)  }  } d\varphi \\ \quad \quad \quad \quad \quad =\prod _{ k=1 }^{ K }{ \left[ \int { \left( \prod _{ t=1 }^{ V }{ { { \varphi  }_{ k,t } }^{ { n }_{ k }^{ (t) } } } *\frac { 1 }{ \triangle (\overrightarrow { \beta  } ) } \prod _{ t=1 }^{ V }{ { { \varphi  }_{ k,t } }^{ { \beta  }_{ t }-1 } }  \right)  } d\overrightarrow { { \varphi  }_{ k } }  \right]  } \\ \quad \quad \quad \quad \quad =\prod _{ k=1 }^{ K }{ \left[ \int { \left( \prod _{ t=1 }^{ V }{ { { \varphi  }_{ k,t } }^{ { n }_{ k }^{ (t) }+{ \beta  }_{ t }-1 } } *\frac { 1 }{ \triangle (\overrightarrow { \beta  } ) }  \right)  } d\overrightarrow { { \varphi  }_{ k } }  \right]  } \\ \quad \quad \quad \quad \quad =\prod _{ k=1 }^{ K }{ \frac { \triangle \left( \overrightarrow { { n }_{ k } } +\overrightarrow { \beta  }  \right)  }{ \triangle (\overrightarrow { \beta  } ) }  } ,\quad 其中\overrightarrow { { n }_{ k } } ={ \left\{ { n }_{ k }^{ (t) } \right\}  }_{ t=1 }^{ V }$  
    至此, 我们发现, 第一项的计算结果, 只和超参数$\overrightarrow { \beta  } $和$\overrightarrow { w } $的单词观测次数有关; 每个主题的单词分布$\overrightarrow { { \varphi  }_{ k } } $已在计算过程中被积分掉.   
    此过程, 属于$Dirichlet-Multinomial$共轭结构. 而$p\left( \overrightarrow { z } |\theta  \right) $的计算也遵循$Dirichlet-Multinomial$共轭结构  
  2. 第二项 : $p\left( \overrightarrow { z } |\overrightarrow { \alpha  }  \right) $的计算也按照第一步的方法. 先假设由超参数$\overrightarrow { \alpha  } $确定的$\overrightarrow { \theta  } $已知, 转而计算   
$$p\left( \overrightarrow { z } |\theta  \right) =\prod _{ t=1 }^{ W }{ p\left( { z }_{ i } \right)  } =\prod _{ m=1 }^{ M }{ \prod _{ k=1 }^{ K }{ { \left( { { \theta  } }_{ m,k } \right)  }^{ { { n }_{ m } }^{ k } } }  } $$其中, ${ { n }_{ m } }^{ k }$是第m篇文档下, 第k号主题的词数(该文档下,topic k被指派给词的计数)  
  此后, 计算$p\left( \overrightarrow { z } |\overrightarrow { \alpha  }  \right) $  
$p\left( \overrightarrow { z } |\overrightarrow { \alpha  }  \right) =\int { p\left( \overrightarrow { z } |\theta  \right) *p\left( \theta |\overrightarrow { \alpha  }  \right)  } d\theta \\ \quad \quad \quad \quad =\int { p\left( \overrightarrow { z } |\theta  \right) *Dirichlet\left( \theta |\overrightarrow { \alpha  }  \right)  } d\theta \\ \quad \quad \quad \quad =\int { \prod _{ m=1 }^{ M }{ \prod _{ k=1 }^{ K }{ { \left( { { \theta  } }_{ m,k } \right)  }^{ { { n }_{ m } }^{ k } } }  } *\frac { 1 }{ \triangle (\overrightarrow { \alpha  } ) } \prod _{ k=1 }^{ K }{ { ({ { \theta  } }_{ m,k }) }^{ { \alpha  }_{ k }-1 } }  } d\theta \\ \quad \quad \quad \quad =\int { \prod _{ m=1 }^{ M }{ \frac { 1 }{ \triangle (\overrightarrow { \alpha  } ) } \prod _{ k=1 }^{ K }{ { \left( { { \theta  } }_{ m,k } \right)  }^{ { { n }_{ m } }^{ k }+{ \alpha  }_{ k }-1 } }  } d\theta  } \\ \quad \quad \quad \quad =\prod _{ m=1 }^{ M }{ \frac { 1 }{ \triangle (\overrightarrow { \alpha  } ) }  } \left( \int { \prod _{ k=1 }^{ K }{ { \left( { { \theta  } }_{ m,k } \right)  }^{ { { n }_{ m } }^{ k }+{ \alpha  }_{ k }-1 } }  } d\theta  \right) \\ \\ \quad \quad \quad \quad =\prod _{ m=1 }^{ M }{ \frac { \triangle (\overrightarrow { { n }_{ m } } +\overrightarrow { \alpha  } ) }{ \triangle (\overrightarrow { \alpha  } ) } \quad  } ,\quad \overrightarrow { { n }_{ m } } ={ \{ { n }_{ m }^{ (k) }\}  }_{ k=1 }^{ K }$  
  其中, $\overrightarrow { { n }_{ m } } $为第$m$篇文档的主体分布, 即$\overrightarrow { { n }_{ m } }=(topic1词数,topic2词数..)$
  
2. 综上所述, $p\left( \overrightarrow { w } ,\overrightarrow { z } |\overrightarrow { \alpha  } ,\overrightarrow { \beta  }  \right) =\prod _{ k=1 }^{ K }{ \frac { \triangle (\overrightarrow { { n }_{ k } } +\overrightarrow { \beta  } ) }{ \triangle (\overrightarrow { \beta  } ) }  } *\prod _{ m=1 }^{ M }{ \frac { \triangle (\overrightarrow { { n }_{ m } } +\overrightarrow { \alpha  } ) }{ \triangle (\overrightarrow { \alpha  } ) } \quad  } $  
其中, $\overrightarrow { { n }_{ m } } $为第$m$篇文档的主体分布, 即$\overrightarrow { { n }_{ m } }=(topic1词数,topic2词数..)$, ( ${ n }_{ k }^{ (t) }$为主题$k$下单词$t$被观测到的次数,$\overrightarrow { { n }_{ m } } $为第$m$篇文档的主体分布 )

#### 三. LDA的Gibbs采样
1. 上面我们已经推导出, LDA整个文本集生成的联合分布概率$p\left( \overrightarrow { w } ,\overrightarrow { z } |\overrightarrow { \alpha  } ,\overrightarrow { \beta  }  \right) $. 要进行Gibbs采样, 需要条件概率$p\left( { z }_{ i }=k|\overrightarrow { { z }_{ \neg i } } ,\overrightarrow { w }  \right) $  
$$\\ p\left( { z }_{ i }=k|\overrightarrow { { z }_{ \neg i } } ,\overrightarrow { w }  \right) =\frac { p\left( \overrightarrow { w } ,\overrightarrow { z }  \right)  }{ p\left( \overrightarrow { { w } } ,\overrightarrow { { z }_{ \neg i } }  \right)  } =\frac { p\left( \overrightarrow { w } ,\overrightarrow { z }  \right)  }{ p\left( w=t,\overrightarrow { { w }_{ \neg i } } ,\overrightarrow { { z }_{ \neg i } }  \right)  } =\frac { p\left( \overrightarrow { w } ,\overrightarrow { z }  \right)  }{ p\left( \overrightarrow { { w }_{ \neg i } } ,\overrightarrow { { z }_{ \neg i } } |{ w }_{ i }=t \right) *p\left( { w }_{ i }=t \right)  } =\frac { p\left( \overrightarrow { w } ,\overrightarrow { z }  \right)  }{ p\left( \overrightarrow { { w }_{ \neg i } } ,\overrightarrow { { z }_{ \neg i } }  \right)  } *\frac { 1 }{ p\left( { w }_{ i }=t \right)  } \\ \propto \frac { p\left( \overrightarrow { w } ,\overrightarrow { z }  \right)  }{ p\left( \overrightarrow { { w }_{ \neg i } } ,\overrightarrow { { z }_{ \neg i } }  \right)  } =\xrightarrow [ \quad \quad 带入LDA文档生成联合概率密度\quad \quad  ]{  } =\frac { \prod _{ k=1 }^{ K }{ \frac { \triangle (\overrightarrow { { n }_{ k } } +\overrightarrow { \beta  } ) }{ \triangle (\overrightarrow { \beta  } ) }  } *\prod _{ m=1 }^{ M }{ \frac { \triangle (\overrightarrow { { n }_{ m } } +\overrightarrow { \alpha  } ) }{ \triangle (\overrightarrow { \alpha  } ) } \quad  }  }{ \prod _{ k=1 }^{ K }{ \frac { \triangle (\overrightarrow { { n }_{ k,\neg i } } +\overrightarrow { \beta  } ) }{ \triangle (\overrightarrow { \beta  } ) }  } *\prod _{ m=1 }^{ M }{ \frac { \triangle (\overrightarrow { { n }_{ m,\neg i } } +\overrightarrow { \alpha  } ) }{ \triangle (\overrightarrow { \alpha  } ) } \quad  }  } =\xrightarrow [ 因为现在计算某个文档中,某个单词的主题概率\quad \quad  ]{ 所以主题号和文档号已确定,\quad 连乘号可消除 } =\frac { \triangle (\overrightarrow { { n }_{ k } } +\overrightarrow { \beta  } )*\triangle (\overrightarrow { { n }_{ m } } +\overrightarrow { \alpha  } ) }{ \triangle (\overrightarrow { { n }_{ k,\neg i } } +\overrightarrow { \beta  } )*\triangle (\overrightarrow { { n }_{ m,\neg i } } +\overrightarrow { \alpha  } ) } $$其中的符号说明 :  
  1. $\overrightarrow { { n }_{ k } } =第k个主题下\left( 第1个单词的个数,第2个单词的个数 ... 第V个单词的个数 \right) $
  2. $\overrightarrow { { n }_{ m } } =第m篇文档中\left( 1号主题的词数,2号主题的词数 ... k号主题的词数\right) $
  3. $\overrightarrow { { n }_{ k,\neg i } } $ 表示除了当前单词i以外, 第k个主题下的V个单词的计数  
  4. $\overrightarrow { { n }_{ m,\neg i } } $ 表示除了当前单词i以外, 第m篇文档中属于K个主题的单词计数  

2. 把$\triangle $展开成$\Gamma$函数
$$p\left( { z }_{ i }=k|\overrightarrow { { z }_{ \neg i } } ,\overrightarrow { w }  \right) =\frac { \triangle (\overrightarrow { { n }_{ k } } +\overrightarrow { \beta  } )*\triangle (\overrightarrow { { n }_{ m } } +\overrightarrow { \alpha  } ) }{ \triangle (\overrightarrow { { n }_{ k,\neg i } } +\overrightarrow { \beta  } )*\triangle (\overrightarrow { { n }_{ m,\neg i } } +\overrightarrow { \alpha  } ) } \\ \because \quad \triangle \left( \overrightarrow { \alpha  }  \right) =\frac { \prod { \Gamma \left( { \alpha  }_{ i } \right)  }  }{ \Gamma \left( \sum { { \alpha  }_{ i } }  \right)  } ,且\frac { \Gamma \left( n \right)  }{ \Gamma \left( n-1 \right)  } =\frac { \left( n-1 \right) ! }{ \left( n-2 \right) ! } =n-1\quad \quad \\ \therefore \quad p\left( { z }_{ i }=k|\overrightarrow { { z }_{ \neg i } } ,\overrightarrow { w }  \right) =\frac { \triangle (\overrightarrow { { n }_{ k } } +\overrightarrow { \beta  } ) }{ \triangle (\overrightarrow { { n }_{ k,\neg i } } +\overrightarrow { \beta  } ) } *\frac { \triangle (\overrightarrow { { n }_{ m } } +\overrightarrow { \alpha  } ) }{ \triangle (\overrightarrow { { n }_{ m,\neg i } } +\overrightarrow { \alpha  } ) } =\frac { ({ n }_{ i }-1)+{ \beta  }_{ i } }{ \sum _{ t=1 }^{ V }{ \left( { n }_{ t,\neg i }+{ \beta  }_{ t } \right)  }  } *\frac { ({ n }_{ k }-1)+{ \alpha  }_{ k } }{ \sum _{ t=1 }^{ K }{ \left( { n }_{ t,\neg i }+{ \alpha  }_{ t } \right)  }  } $$  
结果中:  
  1. $({ n }_{ i }-1)$代表k号主题的第$i$个单词个数-1  
  2. $({ n }_{ k }-1)$代表第m篇文档的第k个主题的单词个数-1  
  3. 最终使用对称参数结果的形式:$$p\left( { z }_{ i }=k|\overrightarrow { { z }_{ \neg i } } ,\overrightarrow { w }  \right) =\left( W_{ i }这个词在第k号主题下的概率 \right) \quad *\quad \left( 第k号主题在第m篇文档中的概率 \right) \\ =\xrightarrow [ 用一个(\alpha ,\beta )代替({ \alpha  }_{ k },{ \beta  }_{ i }) ]{ 使用对称超参数, } =\frac { { n }_{ k,\neg 单词i }^{ (t) }+\beta  }{ \sum _{ t=1 }^{ V }{ \left( { n }_{ k,\neg 单词i }^{ (t) }+{ V\beta  } \right)  }  } *\frac { { n }_{ m,\neg 单词i }^{ (k) }+{ \alpha  } }{ \sum _{ k=1 }^{ K }{ \left( { n }_{ m,\neg 单词i }^{ (k) }+{ K{ \alpha  } } \right)  }  } $$ 
      

#### 四. Gibbs采样后的参数估计
1. 主题采样后, 可以得到参数$\theta $ (doc->topic) 和$\varphi $ (topic->words) 的期望:  
 ${ \theta  }_{ mat }=\left[ \overrightarrow { { \theta  }_{ 1 } } ,\overrightarrow { { \theta  }_{ 2 } } ...\overrightarrow { { \theta  }_{ m } }  \right] $, 其中${ \theta  }_{ i }$是一个K维向量: 元素$${ \theta  }_{ m,k }=\frac { { n }_{ m,k }+{ \alpha  } }{ \sum _{ i=1 }^{ K }{ { n }_{ m,i } } +{ K\alpha  } }  $$  
 ${ \varphi  }_{ mat }=\left[ \overrightarrow { { \varphi  }_{ 1 } } ,\overrightarrow { { \varphi  }_{ 2 } } ...\overrightarrow { { \varphi  }_{ K } }  \right] $, 其中$\overrightarrow { { \varphi  }_{ j } } $是一个V维向量: 元素$${ \varphi  }_{ k,w }=\frac { { n }_{ k,w }+\beta  }{ \sum _{ i=1 }^{ V }{ { n }_{ m,j } } +K\beta  } $$

### 1.2 LDA Gibbs实现